In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Random Number Generator
rng = np.random.default_rng()

## Framework for Simulations

In [ ]:
class Player:
    def __init__(self, actions:int = 0, strategy:np.ndarray = np.empty((0,), dtype=np.float32), size:float = 1, speed:float = 1, fitness:float = 1) -> None:
        self.pastPayoffs = np.empty(shape=(0,))
        self.actions = actions
        assert(strategy.ndim == 1)
        assert(strategy.shape[0] == actions)
        self.strategy = strategy
        if(np.sum(strategy) != 1):
            self.strategy /= max(1., np.sum(strategy))
        self.size = size
        self.speed = speed
        self.fitness = fitness

    def GetAction(self) -> int:
        return rng.choice(self.actions, size=1, p=self.strategy)

    def ExtendPayoffs(self, newPayoffs: np.ndarray) -> None:
        self.pastPayoffs = np.concatenate([self.pastPayoffs, newPayoffs])

    def GetPayoff(self) -> np.ndarray:
        return self.pastPayoffs
    
    def SetSpeed(self, speed:float) -> None:
        self.speed = speed
    
    def SetSize(self, size:float) -> None:
        self.size = size

    def UpdateSpeed(self, delSpeed:float) -> None:
        self.speed += delSpeed

    def UpdateSize(self, delSize:float) -> None:
        self.size += delSize

    def SetFitness(self, fitness: float) -> None:
        self.fitness = fitness
    
    def UpdateFitness(self, delFit: float) -> None:
        self.fitness *= (1+delFit)
        self.fitness = max(self.fitness, 1e-7)


In [ ]:
class Game:
    """ 
    utility: np.ndarray with dimensions nPlayers+1 and shape[i]=player[i].actions and shape[-1]=nPlayers
    """
    def __init__(self, nPlayers:int, strategies:list[np.ndarray], utility:np.ndarray):
        assert(len(strategies) == nPlayers)
        maxActions=0;
        for i in range(nPlayers):
            assert(utility.shape[i] == strategies[i].size)
            maxActions = max(maxActions, strategies[i].size)
        assert(utility.shape[-1] == nPlayers)
        self.Players = [Player(strategies[i].size, strategies[i]) for i in range(nPlayers)]
        self.utility = utility.astype(np.float32)
        strategylengths = [strategies[i].size for i in range(nPlayers)]
        self.tiledProbability = np.ones(shape = [nPlayers, *strategylengths])
        self.allPayoffs = np.empty(shape = [nPlayers, 0])
        self.newPayoffs = np.empty(shape = [nPlayers, 0])
        for i in range(nPlayers):
            x = np.reshape(strategies[i], [1 if j!=i else -1 for j in range(nPlayers)])
            self.tiledProbability[i] *= x

        self.probability = np.prod(self.tiledProbability, axis=0)


    """
    returns payoff of ith player if 0<=i<nPlayers
    return an array of payoff of all players otherwise
    """
    def GetExpectedPayoff(self, i:int) -> np.ndarray:
        u = self.utility.copy()
        u *= np.expand_dims(self.probability, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        if(0<=i and i<len(self.Players)):
            return np.round(payoff[i], 8)
        else:
            return payoff
        
    def GetRandomStrategyProfile(self):
        flat_prob = self.probability.flatten()
        flat_ind = np.random.choice(len(flat_prob), p=flat_prob)
        return np.unravel_index(flat_ind, self.probability.shape)
    
    def GetPayoff(self, actions:tuple, i:int):
        p = self.utility[actions]
        if(0<=i and i<len(self.Players)):
            return p[i]
        else:
            return p

    def PlayGame(self):
        strategyProfile = self.GetRandomStrategyProfile()
        payoffs = self.GetPayoff(strategyProfile, -1)
        payoffs2 = payoffs.reshape((-1,1))
        self.newPayoffs = np.hstack([self.newPayoffs,payoffs2] )
        return payoffs
    
    def GetPayoffForPlayerStrategy(self, player:int, strategy:np.ndarray) -> np.ndarray:
        if(np.sum(strategy) != 1 or strategy.shape != (self.Players[player].actions,)):
            raise ValueError("Not a valid strategy")
        x = np.reshape(strategy, [1 if j!=player else -1 for j in range(len(self.Players))])
        y = self.tiledProbability.copy()
        y[player]=x
        prob = np.prod(y, axis=0)
        u = self.utility.copy()
        u *= np.expand_dims(prob, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        return payoff
    def GetPayoffForStrategy(self, strategy:list[np.ndarray]) -> np.ndarray:
        probability = np.ones(shape = [len(strategy[i]) for i in range(len(strategy))])
        for i in range(len(strategy)):
            x = strategy[i].reshape([1 if i!=j else -1 for j in range(len(strategy))])
            probability *= x
        u = self.utility.copy()
        u = self.utility.copy()
        u *= np.expand_dims(probability, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        return payoff



    def GetPastPayoffs(self, player:int) -> np.ndarray:
        if(self.newPayoffs.size>0):
            for i in range(len(self.Players)):
                self.Players[i].ExtendPayoffs(self.newPayoffs[i])
        self.allPayoffs = np.hstack([self.allPayoffs, self.newPayoffs])
        self.newPayoffs = np.empty(shape=(len(self.Players),0))
        if(0<=player and player<len(self.Players)):
            return self.Players[player].GetPayoff()
        else:
            return self.allPayoffs
            


        

## Chemostat Simulation

### Simulation Variables

In [ ]:
populationSize = 1000
fitnessChange = 0.01
fitnessIncChance = 0.1
fitnessDecChance = 0.11
delChance = 0.2
generations = 1000
parallelSimulations = 10

### Basic Chemostat

In [ ]:
population = [Player(fitness = 1) for i in range(populationSize)]
fitnesses = np.asarray([p.fitness for p in population], dtype=np.float64)
probabilities = fitnesses / np.sum(fitnesses)

fitnessData = np.empty(dtype = np.float64, shape=(0,generations,populationSize))
simFitness = np.empty(dtype = np.float64, shape=(0, populationSize))

fitnessIncreaseCount = 0
fitnessDecreaseCount = 0

higherFitChosenCount = 0
lowerFitChosenCount = 0


for sim in range(1,parallelSimulations+1):
    population = [Player(fitness=1) for i in range(populationSize)]
    fitnesses = np.asarray([p.fitness for p in population], dtype=np.float64)
    probabilities = fitnesses / np.sum(fitnesses)
    simFitness = np.empty(dtype = np.float64, shape=(0, populationSize))
    avgFitness = np.mean(fitnesses)

    delChance /= 2
    fitnessDecChance = fitnessIncChance + delChance

    for i in range(1,generations+1):
        rnd = rng.random(size=(populationSize,))
        parents = rng.choice(range(populationSize), (populationSize,), p=probabilities)
        #print(parents)
        currentPopulation = population
        population = []
        for idx,j in enumerate(parents):
            if(currentPopulation[j].fitness > avgFitness*1.01):
                higherFitChosenCount += 1
            elif(currentPopulation[j].fitness <= avgFitness*0.99):
                lowerFitChosenCount += 1
            population.append(Player(size=currentPopulation[j].size, speed = currentPopulation[j].speed, fitness=currentPopulation[j].fitness))
            if(rnd[j]<fitnessIncChance):
                population[-1].UpdateFitness(fitnessChange)
                fitnessIncreaseCount += 1
            elif(rnd[j]<fitnessIncChance+fitnessDecChance):
                population[-1].UpdateFitness(-fitnessChange)
                fitnessDecreaseCount += 1
            fitnesses[idx] = population[-1].fitness

        probabilities = fitnesses / np.sum(fitnesses)
        avgFitness = np.mean(fitnesses)
        #print(fitnesses)
        simFitness = np.concatenate([simFitness, np.reshape(fitnesses, (1,-1))],axis=0)
    fitnessData = np.concatenate([fitnessData, np.reshape(simFitness, (1,generations, populationSize))], axis=0)
    


### Modified Chemostat

In [ ]:
def UpdateFitness(populationArray: list) -> np.ndarray:
    size = np.empty((populationSize,))
    speed = np.empty((populationSize,))
    for i,p in enumerate(populationArray):
        size[i] = p.size
        speed[i] = p.speed
    fitness = (size-3)*(size-0.5)*(speed-2)*(speed-4)*(size-speed)
    fitness = np.maximum(fitness,-9.99)
    fitness = np.minimum(fitness, 10)
    fitness += 10
    fitness /= 4

    return fitness
    


In [ ]:
population = [Player() for i in range(populationSize)]
fitnesses = np.asarray([p.fitness for p in population], dtype=np.float64)
probabilities = fitnesses / np.sum(fitnesses)

fitnessData = np.empty(dtype = np.float64, shape=(0,generations,populationSize))
simFitness = np.empty(dtype = np.float64, shape=(0, populationSize))

sizeData = np.empty((0, populationSize))
speedData = np.empty((0, populationSize))

for sim in range(parallelSimulations):
    random_n = rng.random(size=(populationSize,2))*2 + 1
    population = [Player(size=random_n[i][0], speed=random_n[i][1]) for i in range(populationSize)]
    fitnesses = np.asarray([p.fitness for p in population], dtype=np.float64)
    probabilities = fitnesses / np.sum(fitnesses)
    simFitness = np.empty(dtype = np.float64, shape=(0, populationSize))
    for i in range(1,generations+1):
        rnd = rng.random(size=(populationSize,2))
        rnd -= 0.5
        parents = rng.choice(population, (populationSize,), p=probabilities)
        population = parents
        for j in range(populationSize):
            population[j].size += rnd[j][0]
            population[j].speed += rnd[j][1]
            population[j].size = max(population[j].size, 0)
            population[j].size = min(population[j].size, 5)
            population[j].speed = max(population[j].speed, 0)
            population[j].speed = min(population[j].speed, 5)
        fitnesses = UpdateFitness(population)
        probabilities = fitnesses / np.sum(fitnesses)
        simFitness = np.concatenate([simFitness, np.reshape(fitnesses, (1,-1))],axis=0)

        sizes = np.ones((populationSize,))
        speeds = np.ones((populationSize,))
        for i in range(populationSize):
            sizes[i] = population[i].size
            speeds[i] = population[i].speed
        
    fitnessData = np.concatenate([fitnessData, np.reshape(simFitness, (1,generations, populationSize))], axis=0)
    sizes = np.ones((populationSize,))
    speeds = np.ones((populationSize,))
    for i in range(populationSize):
        sizes[i] = population[i].size
        speeds[i] = population[i].speed
    sizeData = np.concatenate([sizeData, np.reshape(sizes, (1,-1))], axis=0)
    speedData = np.concatenate([speedData, np.reshape(speeds, (1,-1))], axis=0)


### Visualise

In [ ]:
sortedFitnessData = np.sort(fitnessData)

In [ ]:
print(f"Fitness Increase: {fitnessIncreaseCount}\nFitness Decrease: {fitnessDecreaseCount}")

In [ ]:
print(f"higher fitness chosen: {higherFitChosenCount}\nlower fitness chosen: {lowerFitChosenCount}")

In [ ]:
avgAcrossSims = np.mean(sortedFitnessData, axis=0)
for i in range(10):
    n,bins= np.histogram(avgAcrossSims[int((i+1)*(generations/10)-1)])
    bincentres = 0.5*(bins[1:]+bins[:-1])
    plt.plot(bincentres, n, label=f"gen {(i+1)*generations/10 -1}", alpha = (50+20*i)/250)
plt.legend()

In [ ]:
avgFitnessforGen = np.mean(fitnessData, axis=-1)
averageFitness = np.mean(avgFitnessforGen, axis=0)

In [ ]:
for i in range(parallelSimulations):
    data = np.minimum(avgFitnessforGen[i], 10)
    plt.plot(data, alpha=0.8, label = f"simulation {i+1}")

plt.legend()

#plt.plot(np.minimum(averageFitness,10), color=(0,0,0))
